# SQL Python

Tenemos 3 CSV, debemos crear una base de datos en Dbeaver y usando python ingresar los datos y después realizarnos 5 preguntas al respecto que podamos sacar con consultas que lleven JOIN, GROUP BY o similar

También podemos practicar subsconsultas

In [2]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd

from geopy.geocoders import Nominatim
import dotenv    
import os

from datetime import datetime

Primero voy a cargar los CSV con objetivo de ver que tablas pueda sacar

In [6]:
key = os.getenv("sql")
try:
    conexion = psycopg2.connect(
        database = "laboratorio3",
        user = "postgres",
        password = key,#Esto en un .env
        host = "localhost",
        port = "5432" )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es errónea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")
    else:
        print(f"Ocurrió el error {e}")
    
cursor = conexion.cursor()

# Ahora 5 Preguntas para practicas subconsultas o inner joins

### 1. ¿Cuál es el nombre de los lugares y la categoría asociada para cada municipio?

In [9]:

cursor.execute('''SELECT l.nombre AS Lugar, m.nombre AS municipio, c.nombre AS categoría
                FROM lugares l 
                INNER JOIN municipios m 
                ON l.id_municipio = m.id_municipio
                INNER JOIN categorias c
                ON l.id_categoria = c.id_categoria;''')
pd.DataFrame(cursor.fetchall()).head()



,0,1,2
0,Diverjungla,Ajalvir,Park
1,Parque Infantil,Ajalvir,Park
2,Chiquitin,Ajalvir,Park
3,Isla Perejil,El Álamo,Park
4,Parque de Madera,El Álamo,Park


In [ ]:
cursor.close()
conexion.close()

### 2.¿Cuáles son los registros de tiempo en los que el estado del cielo es 'Muy nuboso' y la fecha es '2024-08-30 19:00:00.000'?

In [10]:
cursor.execute('''SELECT *
                FROM tiempo t 
                WHERE t.id_estado_cielo
                IN (SELECT ec.id_estado_cielo
                    FROM estado_cielo ec
               	    WHERE ec.nombre = 'Muy nuboso'
	                AND t.id_fecha 
                IN (SELECT f.id_fecha
	                FROM fechas f
	                WHERE f.fecha = '2024-08-30 19:00:00.000'));''')

pd.DataFrame(cursor.fetchall()).head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,3118,32,2,25,25,15.0,0.0,0,52,0,70,S,8,70
1,3954,32,2,27,27,31.0,0.0,0,47,0,65,E,20,87
2,4966,32,2,25,25,15.0,0.0,0,52,0,70,S,8,110
3,6199,32,2,25,25,34.0,0.0,0,53,0,65,SE,19,138
